In [1]:
# mydifflib.py
from difflib import SequenceMatcher
#from heapq import nlargest as _nlargest
import numpy as np
import os 
import time 
from heapq import nlargest 


folder = os.getcwd()
folder_db = os.path.join(folder,"app","static","files")
path_civ = os.path.join(folder_db,"lista_key.txt")
path_coords = os.path.join(folder_db,"lista_coords.txt")
civici_tpn = np.loadtxt(path_civ, delimiter = ";" ,dtype='str')
coords = np.loadtxt(path_coords, delimiter = ",")

In [2]:
civici_tpn

array(['CANNAREGIO 3958,', 'CASTELLO 6795,', 'CASTELLO 5332,', ...,
       'PONTE DELLA LATTE,', 'PONTE DE RIALTO,', 'PONTE STORTO,'],
      dtype='<U51')

In [3]:
# Copia-incolla della versione originale
from difflib import SequenceMatcher
from heapq import nlargest as _nlargest

def get_close_matches_indexes_original(word, possibilities, n=3, cutoff=0.6):
    """Use SequenceMatcher to return a list of the indexes of the best 
    "good enough" matches. word is a sequence for which close matches 
    are desired (typically a string).
    possibilities is a list of sequences against which to match word
    (typically a list of strings).
    Optional arg n (default 3) is the maximum number of close matches to
    return.  n must be > 0.
    Optional arg cutoff (default 0.6) is a float in [0, 1].  Possibilities
    that don't score at least that similar to word are ignored.
    """

    if not n >  0:
        raise ValueError("n must be > 0: %r" % (n,))
    if not 0.0 <= cutoff <= 1.0:
        raise ValueError("cutoff must be in [0.0, 1.0]: %r" % (cutoff,))
    result = []
    s = SequenceMatcher()
    s.set_seq2(word)
    for idx, x in enumerate(possibilities):
        s.set_seq1(x)
        if s.real_quick_ratio() >= cutoff and \
           s.quick_ratio() >= cutoff and \
           s.ratio() >= cutoff:
            result.append((s.ratio(), idx))

    # Move the best scorers to head of list
    result = _nlargest(n, result)

    # Strip scores for the best n matches
    #return [x for score, x in result]
    # Modified by Ale
    return result

In [77]:
s = SequenceMatcher()
word1 = "campo de rialto"
word2 = "rialto"
s.set_seq1(word1)
s.set_seq2(word2)
match = s.find_longest_match(0, len(word1), 0, len(word2))
print(word1[match.a:])
matched_words = word1[match.a:].split(" ")
first_matched_word = matched_words[0]
print(match.size / len(first_matched_word))

rialto
1.0


In [80]:
def get_close_matches_indexes(word, possibilities, junk_seq=None, n=3, cutoff=0.5):
    """Use SequenceMatcher to return a list of the indexes of the best
    "good enough" matches. word is a sequence for which close matches
    are desired (typically a string).
    possibilities is a list of sequences against which to match word
    (typically a list of strings).
    Optional arg n (default 3) is the maximum number of close matches to
    return.  n must be > 0.
    Optional arg cutoff (default 0.6) is a float in [0, 1].  Possibilities
    that don't score at least that similar to word are ignored.
    """

    if not n >  0:
        raise ValueError("n must be > 0: %r" % (n,))
    if not 0.0 <= cutoff <= 1.0:
        raise ValueError("cutoff must be in [0.0, 1.0]: %r" % (cutoff,))
    result_ratio = []
    result_idx = []
    # se vogliamo escludere le "junk sequence" (in questo caso lo spazio) SequenceMatcher(lambda x: x==" "),  
    if junk_seq:
        s = SequenceMatcher(isjunk=lambda x: x==junk_seq)
    else:
        s = SequenceMatcher()
    s.set_seq2(word)
    
    for idx, x in enumerate(possibilities):
        s.set_seq1(x)
        
        #print(longest_word, word)
        longest_match = s.find_longest_match(0, len(x), 0, len(word))
        words = x.split(' ')
        longest_word = len(word)
        for c_word in words:
            if len(c_word) > longest_word:
                #print(c_word)
                longest_word = len(c_word)
        our_ratio = (longest_match.size) / longest_word
        
        if our_ratio >= cutoff:
            match = s.find_longest_match(0, len(x), 0, len(word))
            #print(match)
            #print (word[match.a:match.a + match.size])
            result_ratio.append(our_ratio)
            result_idx.append(idx)
    #    if s.real_quick_ratio() >= cutoff and \
    #       s.quick_ratio() >= cutoff and \
    #       s.ratio() >= cutoff:
    #        result_ratio.append(s.ratio())
    #        result_idx.append(idx)

    # Move the best scorers to head of list
    max_idx = []
    if result_ratio:
        ratios = np.asarray(result_ratio)

        result = nlargest(n, result_ratio)
        max_idx = []
        for i in range(n):
            arg_max_idx=np.argmax(result_ratio)
            if result_ratio[arg_max_idx] < cutoff:
                max_idx.append((-1, 0))
            else:
                max_idx.append((result_idx[arg_max_idx], result_ratio[arg_max_idx]))
            #print(result_ratio[arg_max_idx])
            result_ratio[arg_max_idx] = 0
            
    
    # Strip scores for the best n matches
    return max_idx


In [5]:
res = get_close_matches_indexes("gesuiti".upper(), " ", civici_tpn, n=2, cutoff=0.4)
print('Nuova funzione')
[print(civici_tpn[i]) for i in res]

1.0
1.0
Nuova funzione
CAMPO DEI GESUITI,
PONTE DEI GESUITI,


[None, None]

In [6]:
res = get_close_matches_indexes("san polo".upper(), "campo", civici_tpn, n=3, cutoff=0.5)
[civici_tpn[i] for i in res]

1.0
1.0
1.0


['SAN POLO 3059,', 'SAN POLO 557,', 'SAN POLO 1812,']

Creiamo una lista di indirizzi problematici in modo da fare testing di diversi algoritmi. 
servirà un sistema di alias della richiesta(s. -> san) e del database (margarita->margherita)
Si potrebbe togliere le stopwords solo da la parola che andiamo a matchare. E mettere lo spazio come junk.

In [7]:
def correct_name(name):
    # prende la stringa in ingresso e fa delle sostituzioni
    # 0. Eliminare spazi iniziali e finali
    name = name.strip()
    # 1. Sostituzione s.->san
    name = name.replace("s.","san ")
    # 2. Rimozione doppi spazi
    name = name.replace("  "," ")
    # Ritorna stringa corretta
    return name.upper()

In [8]:
import re
from difflib import SequenceMatcher
def search_name(name):
    # in base alla formattazione del nome cerca nel database civici o toponimi
    # prima di tutto correggere il nome
    name = correct_name(name)
    # format del civico: 
    # 1. inizia con un numero (es. "2054 santa marta")
    # 2. finisce con un numero (es. "san polo 1424")
    # 3. finisce con un numero e una lettera (es. "santa croce 1062b" oppure "1062 b" oppure "1062/b")
    # nb se il numero è seguito da una lettera ed è all'inizio del nome, la lettera viene riconosciuta solo se seguita da uno spazio
    format_civico = re.compile("(^\d+([ |/]?\w )?)|(\d+[ |/]?\w?$)")
    is_civico = format_civico.search(name)
    if is_civico:
        # se il nome cercato rientra nel format del civico estrae il numero
        numero = is_civico.group(0)
        # formatta il numero nel format in cui è salvato nel database
        numero_cifra = re.findall(r'\d+',numero)
        numero_lettera = re.findall(r'[a-z]',numero)
        numero = numero_cifra[0]
        if numero_lettera:
            numero += '/' + numero_lettera[0]  
        
        sestiere = name[:is_civico.start()]+name[is_civico.end():]
        sestiere = sestiere.strip() # elimina spazi che possono essersi creati togliendo il numero

        sestieri_list = np.array(['SAN POLO','SANTA CROCE','SAN MARCO','DORSODURO','CANNAREGIO','CASTELLO','GIUDECCA'])
        sestiere_found = get_close_matches(sestiere,sestieri_list,n=1,cutoff=0.6)
        if len(sestiere_found) == 1 or (len(sestiere_found)>1 and SequenceMatcher(None, sestiere, sestiere_found[0]).ratio())>0.85):
            # concatena sestiere e numero
            name2search = sestiere_found[0] + " " + numero
            # TODO lista INDIRIZZI
            list_of_possibilities = lista_indirizzi
        else:
            name2search = sestiere + " " + numero
            list_of_possibilities = lista_denominazioni
        results = get_close_matches_indexes_original(name2search,list_of_possibilities,n=5,cutoff=0.4)
        
    else:
        # TODO cerca usando db toponimi 

SyntaxError: unexpected EOF while parsing (<ipython-input-8-ea5c02fc85d3>, line 32)

In [90]:
import re
name = "1424 san polo"
format_civico = re.compile("(^\d+([ |/]?\w )?)|(\d+[ |/]?\w?$)")
is_civico = format_civico.search(name)
is_civico

<_sre.SRE_Match object; span=(0, 4), match='1424'>

In [8]:
from difflib import get_close_matches, SequenceMatcher
search = "san marlo"
str2search = search.upper()
sestieri_list = np.array(['SAN POLO','SANTA CROCE','SAN MARCO','DORSODURO','CANNAREGIO','CASTELLO','GIUDECCA'])
print('GET_CLOSE_MATCHES')
sestiere_found = get_close_matches(str2search,sestieri_list,n=5,cutoff=0.7)
for s in sestiere_found:
    print(search + " > " + s + " - " + str(SequenceMatcher(None, str2search, s).ratio()))
print('GET_CLOSE_MATCHES_INDEXES')
matches = get_close_matches_indexes_original(str2search,sestieri_list,n=5,cutoff=0.7)
for s,m in matches:
    print(search + " > " + sestieri_list[m] + " - " + str(s))


GET_CLOSE_MATCHES
san marlo > SAN MARCO - 0.8888888888888888
san marlo > SAN POLO - 0.7058823529411765
GET_CLOSE_MATCHES_INDEXES
san marlo > SAN MARCO - 0.8888888888888888
san marlo > SAN POLO - 0.7058823529411765


In [82]:
from fuzzywuzzy import fuzz, process
import time
search = 'rialto'
str2search = search.upper()

# CLOSE MATCHES
t1 = time.time()
matches = get_close_matches(str2search,civici_tpn,n=5,cutoff=0.6)
print('\nGET_CLOSE_MATCHES - {timed} seconds'.format(timed=time.time()-t1))
for m in matches:
    print(search + " > " + m + " - " + str(SequenceMatcher(None, str2search, m).ratio()))

# MODIFIED INDICES
t1 = time.time()
matches = get_close_matches_indexes(str2search,"", civici_tpn,n=5,cutoff=0.6)
print('\nGET_CLOSE_MATCHES_INDEXES_MODIFIED - {timed} seconds'.format(timed=time.time()-t1))
for m in matches:
    if m[0] > 0:
        print(search + " > " + str(civici_tpn[m[0]]) + " - " + str(m[1]))
    else:
        print(search + " > " + str('niente') + " - " + str(m[1]))
    
# INDICES ORIGINAL
t1 = time.time()
matches = get_close_matches_indexes_original(str2search,civici_tpn,n=5,cutoff=0.3)
print('\nGET_CLOSE_MATCHES_INDEXES_ORIGINAL - {timed} seconds'.format(timed=time.time()-t1))
for s,m in matches:
    print(search + " > " + civici_tpn[m] + " - " + str(s))
    
# FUZZY EXTRACT
t1 = time.time()
matches = process.extract(str2search,civici_tpn,limit=5)
print('\nFUZZY RATIO - extract - {timed} seconds'.format(timed=time.time()-t1))
for m,s in matches:
    print(search + " > " + m + " - " + str(s))
    
# FUZZY EXTRACTBESTS
t1 = time.time()
matches = process.extractBests(str2search,civici_tpn,score_cutoff=60,limit=5)
print('\nFUZZY RATIO - extractBests - {timed} seconds'.format(timed=time.time()-t1))
for m,s in matches:
    print(search + " > " + m + " - " + str(s))



GET_CLOSE_MATCHES - 0.14178085327148438 seconds
rialto > RIELO, - 0.6666666666666666
rialto > RIELO, - 0.6666666666666666

GET_CLOSE_MATCHES_INDEXES_MODIFIED - 0.479686975479126 seconds
rialto > CAMPO RIALTO NOVO, - 1.0
rialto > PONTE DE RIALTO, - 0.8571428571428571
rialto > niente - 0
rialto > niente - 0
rialto > niente - 0

GET_CLOSE_MATCHES_INDEXES_ORIGINAL - 0.14350366592407227 seconds
rialto > RIELO, - 0.6666666666666666
rialto > RIELO, - 0.6666666666666666

FUZZY RATIO - extract - 3.1217525005340576 seconds
rialto > SOTOPORTEGO DE RIALTO, - 90
rialto > CAMPO RIALTO NOVO, - 90
rialto > PONTE DE RIALTO, - 90
rialto > RIELO, - 73
rialto > RIELO, - 73

FUZZY RATIO - extractBests - 3.0011165142059326 seconds
rialto > SOTOPORTEGO DE RIALTO, - 90
rialto > CAMPO RIALTO NOVO, - 90
rialto > PONTE DE RIALTO, - 90
rialto > RIELO, - 73
rialto > RIELO, - 73


In [13]:
matches
for m,s in matches:
    print(m,str(s))

SOTOPORTEGO DE RIALTO, 90
CAMPO RIALTO NOVO, 90
PONTE DE RIALTO, 90
RIELO, 73
RIELO, 73


In [81]:
from fuzzywuzzy import fuzz, process
import time
searches = ['rialto', 'santa margherita', 'baia del re', '2054 santa marta', 'cannaregio 3727/b',
                               'erbaria', 'campo san polo', 'san polo 1414']
for search in searches:
    str2search = search.upper()

    # CLOSE MATCHES
    t1 = time.time()
    matches = get_close_matches(str2search,civici_tpn,n=5,cutoff=0.6)
    print('\nGET_CLOSE_MATCHES - {timed} seconds'.format(timed=time.time()-t1))
    for m in matches:
        print(search + " > " + m + " - " + str(SequenceMatcher(None, str2search, m).ratio()))

    # MODIFIED INDICES
    t1 = time.time()
    matches = get_close_matches_indexes(str2search,"", civici_tpn,n=5,cutoff=0.51)
    print('\nGET_CLOSE_MATCHES_INDEXES_MODIFIED - {timed} seconds'.format(timed=time.time()-t1))
    for m in matches:
        if m[0] > 0:
            print(search + " > " + str(m[0]) + ": " + str(civici_tpn[m[0]]) + " - " + str(m[1]))
        else:
            print(search + " > " + str(m[0]) + ": " + str('niente') + " - " + str(m[1]))

    # INDICES ORIGINAL
    t1 = time.time()
    matches = get_close_matches_indexes_original(str2search,civici_tpn,n=5,cutoff=0.6)
    print('\nGET_CLOSE_MATCHES_INDEXES_ORIGINAL - {timed} seconds'.format(timed=time.time()-t1))
    for s,m in matches:
        print(search + " > " + str(m) + ": " + civici_tpn[m] + " - " + str(s))

    # FUZZY EXTRACT
    t1 = time.time()
    matches = process.extract(str2search,civici_tpn,limit=5)
    print('\nFUZZY RATIO - extract - {timed} seconds'.format(timed=time.time()-t1))
    for m,s in matches:
        print(search + " > " + m + " - " + str(s))

    # FUZZY EXTRACTBESTS
    t1 = time.time()
    matches = process.extractBests(str2search,civici_tpn,score_cutoff=60,limit=5)
    print('\nFUZZY RATIO - extractBests - {timed} seconds'.format(timed=time.time()-t1))
    for m,s in matches:
        print(search + " > " + m + " - " + str(s))



GET_CLOSE_MATCHES - 0.12313246726989746 seconds
rialto > RIELO, - 0.6666666666666666
rialto > RIELO, - 0.6666666666666666

GET_CLOSE_MATCHES_INDEXES_MODIFIED - 0.4342656135559082 seconds
rialto > 37486: CAMPO RIALTO NOVO, - 1.0
rialto > 38372: PONTE DE RIALTO, - 0.8571428571428571
rialto > 37457: SOTOPORTEGO DE RIALTO, - 0.5454545454545454
rialto > -1: niente - 0
rialto > -1: niente - 0

GET_CLOSE_MATCHES_INDEXES_ORIGINAL - 0.14232802391052246 seconds
rialto > 38245: RIELO, - 0.6666666666666666
rialto > 36211: RIELO, - 0.6666666666666666

FUZZY RATIO - extract - 2.982512950897217 seconds
rialto > SOTOPORTEGO DE RIALTO, - 90
rialto > CAMPO RIALTO NOVO, - 90
rialto > PONTE DE RIALTO, - 90
rialto > RIELO, - 73
rialto > RIELO, - 73

FUZZY RATIO - extractBests - 3.168370008468628 seconds
rialto > SOTOPORTEGO DE RIALTO, - 90
rialto > CAMPO RIALTO NOVO, - 90
rialto > PONTE DE RIALTO, - 90
rialto > RIELO, - 73
rialto > RIELO, - 73

GET_CLOSE_MATCHES - 0.33636975288391113 seconds
santa margher

In [85]:
## TEST FUNCTION
list_of_challenging_searches = ['rialto', 'santa margherita', 'baia del re', '2054 santa marta', 'cannaregio 3727/b',
                               'erbaria', 'campo san polo', 'san polo 1414']
# per ogni ricerca abbiamo 3 risultati e diamo un punteggio in base a cosa ogni metodo trova
list_of_wanted_result = [[37486, 38372, 37457], []]

functions_to_test = ['get_close_matches_indexes', 'get_close_matches_indexes_original']

n = 3
cutoff = 0.5

for function_to_test in functions_to_test:
    
    punti = 0
    
    # loop through searches
    for i in range(len(list_of_challenging_searches)):
        
        challenging_search = list_of_challenging_searches[i]
        desired_results = list_of_wanted_result[i]
        indices, ratio = function_to_test(challenging_search, civici_tpn, n, cutoff)
        punti_for_this_search = calculate_points(indices, ratio, desired_results)
        print("{pt} obtained for this search ({search})".format(pt=punti_for_this_search, search=challenging_search))
        punti += punti_for_this_search
        
    print("method {met} obtained {x} pts".format(met=function_to_test, x=punti))
    

TypeError: 'str' object is not callable